## Step 1)

Import the necessary packages, enter your api key, and read the input file. 


In [1]:
import folium
import numpy as np
import pandas as pd
from geopy.geocoders import MapBox
from ipywidgets import FloatProgress
from IPython.display import display

# You need this to query mapbox
api_key="pk.eyJ1IjoianVuZXNwYWNlYm9vdHMiLCJhIjoiY2twY3g4aXloMWFlcDJzbXN3aG95aG5uZiJ9.mFiJt0MIfL1MiJ2rB2xhKQ"

# Sepcifiy the location of your data
input_filename='Locations.csv'
output_filename = 'Output.csv'


if api_key == "":
    print('Please enter your API key to continue!')
else:
    Data = pd.read_csv('Input/'+input_filename)
Data.head()

,Location
0,"Vancouver, British Columbia, Canada"
1,I am on Bowen Island!
2,"Kamloops BC, Canada"
3,"Vancouver, British Columbia, Canada"
4,"Ottawa, Ontario, Canada"


## Step 2)

Specify the geocode request format

In [3]:
location_columns = ['Location']#['Address','City','Prov']#/Location','State/Province','Postal Code','Country']
## Use this to add exra info that isn't specified (eg. missing country/province)
additional_info = ''#, Canada'

attempt = ', '.join(x for x in location_columns)+ additional_info

print('attempt format')
print(attempt)

attempt format
Location


## Step 3)

Geocode the points

In [4]:
Data['latitude'] = np.nan
Data['longitude'] = np.nan
Data['geocoding_Notes'] = ''

prog = FloatProgress(min=0, max=100,description='Progress:')
prog.value=0
display(prog)


geolocator = MapBox(api_key=api_key)
for index, row in Data.iterrows():
    row[location_columns]=row[location_columns].fillna('')
    attempt = ' '.join(row[x] for x in location_columns) +' ' + additional_info
    prog.value=(index+1)/len(Data)*100
    try:
        g = geolocator.geocode(attempt,timeout=3)
        Data.loc[Data.index==index,['latitude','longitude']]=g.latitude,g.longitude
    except:
        Data.loc[Data.index==index,'geocoding_Notes']='Geocoding Failed'
        pass
Data.to_csv('Output/'+output_filename)
Failures = Data.loc[Data['geocoding_Notes']=='Geocoding Failed']

print('Geocoding Done')
print()
if Failures.count()[location_columns[0]]>0:
    print('Filed to Geocode: ',Failures.count()[location_columns[0]], ' records.')
    print(Failures)
else:
    print('All Records Geocoded')

FloatProgress(value=0.0, description='Progress:')

Geocoding Done

Filed to Geocode:  3  records.
                                             Location  latitude  longitude  \
11  I am currently In Vancouver, British Columbia,...       NaN        NaN   
20                              Vancouver/ Nanaimo BC       NaN        NaN   
27                          Meguro-ku / Tokyo / Japan       NaN        NaN   

     geocoding_Notes  
11  Geocoding Failed  
20  Geocoding Failed  
27  Geocoding Failed  


## Step 4)

Inspect the results.  Create a web map to display the points.

In [5]:
# your mapbox api key also gives you access to the mapbox satelite basemap
token = api_key
tileurl = 'https://api.mapbox.com/v4/mapbox.satellite/{z}/{x}/{y}@2x.png?access_token=' + str(token)

Map = folium.Map(location=[53, -125],zoom_start=3, tiles=tileurl, attr='Mapbox')

for index, row in Data.loc[Data['geocoding_Notes']!='Geocoding Failed'].iterrows():
    row[location_columns]=row[location_columns].fillna('')
    attempt = ' '.join(row[x] for x in location_columns) +' ' + additional_info
    folium.CircleMarker(location=[row['latitude'],row['longitude']],
                        popup=attempt,
                        fill_color='blue',
                        color='black'
                       ).add_to(Map)
Map